In [1]:
# Import the functions used in this project
import awesome_functions as af

# Pretty display for notebooks
%matplotlib inline

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
submission = pd.read_csv("../asset/sample_submission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("submission  data : ", submission.shape)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle as pkl
slack_url = pkl.load(open("Slack_url/send_url.pickle", "rb"))

import xgboost

# 원본을 유지하기 위해서 카피
df_train = train.copy()
df_test = test.copy()
df_submission = submission.copy()

Train data :  (647054, 7)
Test  data :  (653646, 6)
submission  data :  (95674, 39)


In [2]:
df_train_dd = pd.read_csv("Feature_matrix/df_train_dd_201807291831.csv")
df_train_fl = pd.read_csv("Feature_matrix/df_train_fl_201807291845.csv")

In [3]:
X, y = af.get_df_to_fit(df_train_dd)

In [4]:
df_train_dd.head()

,VisitNumber,TripType,Return,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,...,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,5,999,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,7,30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,8,26,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,9,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,10,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [18]:
xgb_model = xgboost.XGBClassifier(n_estimators = 250, max_depth = 2, nthread=4)

In [68]:
xgb_model_10_17 = xgboost.XGBClassifier(n_estimators = 10, max_depth = 17, nthread=4)

In [69]:
xgb_model_10_17.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=17, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=4, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [70]:
y_pred = xgb_model_10_17.predict(X)
af.getAccuracy(y, y_pred, len(X))

##### Accuracy : 0.722

In [71]:
y_pred_proba = xgb_model_10_17.predict_proba(X_test[X.columns])
submit_df = af.make_submission_df(submission, y_pred_proba)
af.saveDataFrameToCsv(submit_df, "submit_xgb_10_17", is_submission_df=True)

In [19]:
xgb_model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=250,
       n_jobs=1, nthread=4, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [21]:
df_test_dd = pd.read_csv("Feature_matrix/df_test_dd_201807291855.csv")

In [23]:
X_test = af.get_df_to_fit(df_test_dd, is_test_df=True)

In [20]:
y_pred = xgb_model.predict(X)

In [22]:
af.getAccuracy(y, y_pred, len(X))

##### Accuracy : 0.6565

In [32]:
y_pred_proba = xgb_model.predict_proba(X_test[X.columns])

In [33]:
submit_df = af.make_submission_df(submission, y_pred_proba)

In [34]:
af.saveDataFrameToCsv(submit_df, "submit_xgb_dd", is_submission_df=True)

In [42]:
def get_df_to_fit(df, is_test_df = False):
    
    # test 모델은 TripType이 없으므로, X만 반환한다.
    if is_test_df:
        return df.drop(["VisitNumber"], axis = 1)
    
    # model에 fit할 때 사용할 X, y를 반환한다. (X, y로 받아줘야 한다.)
    return df.drop(["TripType", "VisitNumber"], axis = 1), df["TripType"].reset_index()

In [48]:
X_train_fl = df_train_fl.drop(["TripType", "VisitNumber"], axis = 1)

In [62]:
df_test_fl = pd.read_csv("Feature_matrix/df_test_fl_201807291908.csv")

In [63]:
X_test_fl = df_test_fl.drop(["VisitNumber"], axis = 1)

In [65]:
X_test_fl_pca = pca.transform(X_test_fl)

In [49]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
pca.fit(np.array(X_train_fl))

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [50]:
X_train_fl_pca = pca.transform(X_train_fl)

In [52]:
xgb_model_fl = xgboost.XGBClassifier(n_estimators = 10, max_depth = 17, nthread=4)

In [53]:
xgb_model_fl.fit(X_train_fl_pca, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=17, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=4, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [54]:
# cv = KFold(10)
# scores = np.zeros(10)
# for i, (train_index, test_index) in enumerate(cv.split(X)):
#     X_train = X[train_index]
#     y_train = y[train_index]
#     X_test = X[test_index]
#     y_test = y[test_index]
#     xgb_model.fit(X_train, y_train)
#     y_pred = xgb_model.predict(X_test)
#     scores[i] = r2_score(y_test, y_pred)

# scores

In [55]:
af.sendSlackDmndSlackDm(slack_url, "xgb fl pca model fit 완료")

In [60]:
y_pred_fl_pca = xgb_model_fl.predict(X_train_fl_pca)

In [61]:
af.getAccuracy(y, y_pred_fl_pca, len(X))

##### Accuracy : 0.8688

In [66]:
y_pred_fl_pca_proba = xgb_model_fl.predict_proba(X_test_fl_pca)

In [67]:
submit_df = af.make_submission_df(submission, y_pred_fl_pca_proba)
af.saveDataFrameToCsv(submit_df, "submit_xgb_fl_pca", is_submission_df=True)

In [ ]:
from keras.datasets import mnist
from keras.utils import np_utils

(X_train0, y_train0), (X_test0, y_test0) = mnist.load_data()

X_train = X_train0.reshape(60000, 784).astype('float32') / 255.0
X_test = X_test0.reshape(10000, 784).astype('float32') / 255.0
Y_train = np_utils.to_categorical(y_train0, 10)
Y_test = np_utils.to_categorical(y_test0, 10)

from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

np.random.seed(0)
model0 = Sequential()
model0.add(Dense(15, input_dim=784, activation="sigmoid"))
model0.add(Dense(10, activation="sigmoid"))
model0.compile(optimizer=SGD(lr=0.2), loss='mean_squared_error', metrics=["accuracy"])

%%time
hist0 = model0.fit(X_train, Y_train, epochs=30, batch_size=100, validation_data=(X_test, Y_test), verbose=0)